## Springboard Data Science Track Capstone Project 1
### Histopathologic Cancer Detection
### by Morgan Fry
### Baseline Model

Now that we have done the data wrangling and some exploratory data analysis we are going to  try to build a baseline image classification model based. We will try a logistic regression to begin with.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
%matplotlib inline

from sklearn.utils import shuffle
import pickle
import sys
import cv2
import seaborn as snsb0811f6beb7d82331f7f0f40cd75af0820c5e02ebdadd8b8
from sklearn.utils import shuffle

import scipy as sp
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

from time import time
import logging

In [2]:
#load data 
y_d=pickle.load(open("saved/y.p","rb"))
X_d=pickle.load(open("saved/X.p","rb"))

In [3]:
#organize data into dataframes
#to save computational time at this point, we'll use the cropped images
X_crop_d={}

for each in X_d.items():
    X_crop_d.update({each[0]:each[1][31:63,31:63].flatten()})
  #  X_v_d.update({each[0]:each[1].flatten()})


In [4]:
X_crop_df=pd.DataFrame.from_dict(X_crop_d,orient='index')
#X_vect_df=pd.DataFrame.from_dict(X_vect_d,orient='index')

In [5]:
#2 dataframes, one with cropped images, both with id's and labels
Xy_crop_df=X_crop_df.merge(y_d,right_on='id',left_index=True)
#Xy_df=X_vect_df.merge(y_d,right_on='id',left_index=True)

In [6]:
Xy_crop_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3064,3065,3066,3067,3068,3069,3070,3071,id,label
0,244,243,245,245,247,248,245,247,248,244,...,249,245,248,247,243,245,244,240,f38a6374c348f90b587e046aac6079959adf3835,0
1,253,252,254,250,247,249,234,231,233,255,...,249,249,249,249,249,248,250,250,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,184,174,187,195,162,189,177,148,174,173,...,199,226,168,160,183,199,193,212,755db6279dae599ebb4d39a9123cce439965282d,0
3,173,106,173,218,139,206,172,98,158,191,...,85,145,177,106,169,191,118,186,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,74,36,66,82,39,96,212,178,242,119,...,20,80,105,64,121,77,27,81,068aba587a4950175d04c680d38943fd488d6a9d,0


In [7]:
#now the dataset is organized we'll split it and scale it
X_train, X_test, y_train, y_test = train_test_split(Xy_crop_df.iloc[:,:3071].values, 
                                              Xy_crop_df.label.values)


In [8]:
#scale the data
scaler = StandardScaler() 
scaler.fit(X_train)  
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test) 

In [ ]:
clf = LogisticRegression(n_jobs=-1)
# Fit the model on the trainng data.
clf.fit(X_train, y_train)


In [14]:
print('Classification Report (train)')
print(classification_report(y_train,clf.predict(X_train)))
print('Classification Report (test)')
print(classification_report(y_test,clf.predict(X_test)))
print('Confusion Matrix:')
print(confusion_matrix(y_test,clf.predict(X_test)))

Classification Report (train)
              precision    recall  f1-score   support

           0       0.72      0.83      0.77     98161
           1       0.68      0.53      0.60     66857

    accuracy                           0.71    165018
   macro avg       0.70      0.68      0.68    165018
weighted avg       0.71      0.71      0.70    165018

Classification Report (test)
              precision    recall  f1-score   support

           0       0.72      0.83      0.77     32747
           1       0.67      0.52      0.59     22260

    accuracy                           0.70     55007
   macro avg       0.70      0.68      0.68     55007
weighted avg       0.70      0.70      0.70     55007

Confusion Matrix:
[[27027  5720]
 [10577 11683]]


A simple logistic regression does an inadequate job for this problem. We are looking to identify samples with cancerous cells in images. What measure shoul we be aiming for? There are a few factors to consider:
* Both false negatives and false positives matter. 
More false positives among a population in which the positive condition is rare means that even with a high accuracy a positive result can still mean a low chance of the condition tested for. More false negatives are even more important as that can mean sick people do not get treatment they need.
* There are about 50% more negative than positive samples in this dataset, so as you see above there is much better recall for negative samples than positive. 

Given these factors using the F1 score for the Positive case is the proper metric to judge the classifier by at this point.

Looking at the results, we can tell a few things:
* The F1 score for Positive cases over the test set is .59, for Negatives .77. 
* Accordingly, the confusion matrix shows about twice as many false negatives (10577) as false positives (5720)
* The numbers are all similar for the train and test set, indicating the model isn't overfit.

#### We need a rather better model than this. We need to see scores of .95 or better for a classifier to be useful in this field. We might be able to eek out a little better results with tuning some of the parameters of the logistic regression, but ultimately we need to find a more robust strategy. Maybe a SVM or random forest or ultimately a neural net.